In [ ]:
import os

import numpy as np
import cv2
from matplotlib import pyplot as plt
import importlib

import config


# Load IR data CSV

In [ ]:
csv_file_path = os.path.abspath(os.path.join('.', 'example_data', 'one_person_close.csv'))
print(f"csv_file_path={csv_file_path}")

with open(csv_file_path, 'r') as file:
    lines = file.readlines()
    
lines = [x.strip() for x in lines] 


print(f"{lines[0]}\n {lines[1][:90]} \n {lines[2][:90]}")

frame_lines = lines[1:]
number_of_ir_frames = len(frame_lines)
print(f"Number of frames: {number_of_ir_frames}")


In [ ]:
def get_ir_frame_data(frame_index):
    line_parts = frame_lines[frame_index].split(',')
    frame_data_str = line_parts[2:] 
    frame_data_1d = [float(x) for x in frame_data_str]
    frame_2d = np.reshape(frame_data_1d, config.IR_CAMERA_RESOLUTION)
    return frame_2d




# Show some frames

In [ ]:
for frame_index in [0, number_of_ir_frames//3, number_of_ir_frames//2, number_of_ir_frames-1]:
    frame_2d = get_ir_frame_data(frame_index)
    plt.figure()
    plt.imshow(frame_2d)

# Extrapolation of the frame


In [ ]:
frame_index = 3
frame_2d = get_ir_frame_data(3)


def get_extrapolated_ir_frame(frame_2d, multiplier = 4, interpolation=cv2.INTER_LINEAR):
    new_size = (frame_2d.shape[1] * multiplier, frame_2d.shape[0] * multiplier)
    frame_2d_resized = cv2.resize(src=frame_2d, dsize=new_size, interpolation=interpolation)
    return frame_2d_resized


for multiplier in [1, 2, 4, 8]:
    print(f"Size multiplied by {multiplier} (linear and cubic) interpolation")
    frame_2d_rsized_linear = get_extrapolated_ir_frame(frame_2d, multiplier=multiplier, interpolation=cv2.INTER_LINEAR)
    frame_2d_rsized_cubic = get_extrapolated_ir_frame(frame_2d, multiplier=multiplier, interpolation=cv2.INTER_CUBIC)
    
    plt.figure()
    plt.imshow(frame_2d_rsized_linear)
    plt.figure()
    plt.imshow(frame_2d_rsized_cubic)
    plt.show()



In [ ]:
rgb_video_path = "/media/przemek/data/Projects/ir_vision/data/data__24_03_2021__20_26_41/000__20_26_41/rgb.mjpeg"
print(f"csv_file_path={csv_file_path}, rgb_video_path={rgb_video_path}")


In [ ]:
cap = cv2.VideoCapture(rgb_video_path)

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )

In [ ]:

rgb_video_cap = cv2.VideoCapture(rgb_video_path)
if not rgb_video_cap.isOpened():
    raise Exception("video open failed")
    
    
number_of_rgb_frames = 0
while True:
    flag, frame = rgb_video_cap.read()
    if flag:
        # The frame is ready and already captured
        # cv2.imshow('video', frame)
        number_of_rgb_frames += 1
        # print(f"Frame no {number_of_rgb_frames}")
        if number_of_rgb_frames % 50 == 0:
            print(f"Reading frame no {number_of_rgb_frames}")
    else:
        print("Frames finished?")
        break
        

rgb_video_cap = cv2.VideoCapture(rgb_video_path)
if not rgb_video_cap.isOpened():
    raise Exception("video open failed")


rgb_frames_for_every_ir_frame = number_of_rgb_frames / number_of_ir_frames

print(f"Total RGB frames: {number_of_rgb_frames}. rgb_frames_for_every_ir_frame: {rgb_frames_for_every_ir_frame:.3f}")


In [ ]:


def get_rgb_frame_for_ir_frame(ir_frame_index):
    global rgb_video_cap
    rgb_frame_index = int(ir_frame_index * rgb_frames_for_every_ir_frame)
    assert rgb_frame_index <  number_of_rgb_frames
    
    current_position = rgb_video_cap.get(cv2.CAP_PROP_POS_FRAMES)
    print(f"Preparing to read RGB frame no {rgb_frame_index}. Current position in video: {current_position}")

    if current_position > rgb_frame_index:
        rgb_video_cap = cv2.VideoCapture(rgb_video_path)
        if not rgb_video_cap.isOpened():
            raise Exception("video open failed")

    frame = None
    while rgb_video_cap.get(cv2.CAP_PROP_POS_FRAMES) != rgb_frame_index:
        ret, frame = rgb_video_cap.read()
        if not ret:
            raise Exception(f"Failed to read frame no {rgb_frame_index}")
    return frame


# show some frames
plt.figure()
plt.imshow(get_rgb_frame_for_ir_frame(1))

plt.figure()
plt.imshow(get_rgb_frame_for_ir_frame(number_of_ir_frames // 2))

In [ ]:
for ir_frame_index in [22, 555, 666, 777, 888]:

    ir_frame_2d = get_ir_frame_data(ir_frame_index)
    ir_frame_2d_extrapolated = get_extrapolated_ir_frame(ir_frame_2d)
    rgb_frame_2d = get_rgb_frame_for_ir_frame(ir_frame_index)

    plt.figure()
    plt.imshow(ir_frame_2d_extrapolated)


    plt.figure()
    plt.imshow(rgb_frame_2d)